# 동적크롤링 과제3(이준목)

- 스타벅스 매장 원하는 지역의 매장 정보는 동적 크롤링으로 가져오자 (전국 매장 DB, CSV저장)

In [2]:
! pip install selenium

In [3]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.request as req

In [ ]:
# 검색 정보
url = "https://www.starbucks.co.kr/store/store_map.do"
sidos = []

# 크롬 브라우저 생성 및 오픈
driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)
element = driver.find_element("css selector","header.loca_search > h3 > a")
element.click()

time.sleep(1)
elements = driver.find_elements("css selector","div.loca_step1_cont > ul > li > a")
for element in elements:
    time.sleep(1)
    sidos.append(element.text)

for i in range(len(sidos)):
    driver.get(url)
    time.sleep(1)
    element = driver.find_element("css selector","header.loca_search > h3 > a")
    element.click()
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, sidos[i])))
    element = driver.find_element(By.LINK_TEXT, sidos[i])
    element.click()
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "전체")))
    element = driver.find_element(By.LINK_TEXT, "전체")
    element.click()

    #mCSB_3_container > ul
    store_list = driver.find_elements("css selector", "#mCSB_3_container > ul > li")
    time.sleep(2)


AttributeError: 'list' object has no attribute 'text'

In [ ]:
# 검색 정보
url = "https://www.starbucks.co.kr/store/store_map.do"
sidos = []
googuns = []

# 검색할 지역 이름 리스트로 추가
while True:
    sido = input(" 시/도 를 입력하세요(2글자) ex) 서울/부산/경남/세종 (종료하려면 'exit' 입력) : ")
    if sido.lower() == 'exit':  # 'exit' 입력하면 종료
        break
    elif sido.lower() == '세종':
        continue
    sidos.append(sido)  # 리스트에 입력값 추가
    googun = input(" 구/군 을 입력하세요 ex) 전체/중구/연제구/서귀포시 : ")
    googuns.append(googun)

# 크롬 브라우저 생성 및 오픈
driver = webdriver.Chrome()

for i in range(len(sidos)):
    driver.get(url)
    time.sleep(0.5)

    # 지역 검색 버튼 클릭
    #container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a
    try:
        time.sleep(1)
        element = driver.find_element("css selector","header.loca_search > h3 > a")
        element.click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, sidos[i])))
        element = driver.find_element(By.LINK_TEXT, sidos[i])
        element.click()
        if googuns[i] !='':
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, googuns[i])))
            element = driver.find_element(By.LINK_TEXT, googuns[i])
            element.click() 
    except:
        print("지역 검색 버튼 찾을 수 없음")

    # 검색 결과 가져오기
    time.sleep(3)

    store_list = driver.find_elements("css selector", "#mCSB_3_container > ul > li")
    time.sleep(3)

    count =0
    temp_list=[]

    for store in store_list:
        count+=1
        lat = store.get_attribute('data-lat')
        long = store.get_attribute('data-long')
        name = store.get_attribute('data-name')
        storeinfo = store.find_element(By.TAG_NAME,"p")
        arr_storeinfo=storeinfo.text.split("\n")
        if(len(arr_storeinfo)==2):
            address = arr_storeinfo[0]
            phone_number = arr_storeinfo[1]
        else:
            address = '-'
            phone_number = '-'
        
        temp_list.append((name, lat, long, address, phone_number))

        total = len(store_list)

        if count != total and count %3==0:
            driver.execute_script("var su=arguments[0];\
                                  var dom=document.querySelectorAll('#mCSB_3_container > ul > li')[su];\
                                  dom.scrollIntoView();", count)

print(temp_list)

14
